# Third Notebook for the Capstone assignment for Week 3

## Exploring boroughs and neighbourhoods within Toronto

## In this notebook, we segment and cluster the neighbourhoods of Toronto. We will work on the borough of Scarborough for the purposes of this notebook

First we import all necessary libraries

In [1]:
import numpy as np 
import pandas as pd 
import json
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



Then we import our Toronto data

In [62]:
toronto_data = pd.read_csv('toronto_data_2.csv')
toronto_data.drop('Unnamed: 0', axis = 1, inplace = True)
toronto_data.head()

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Then we get our map of Toronto

In [3]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent='explore_toronto')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


Then we draw our map

In [4]:
toronto_map = folium.Map(location = [latitude, longitude], zoom_start = 10)

for lat, lng, neighbourhood, borough in zip(
    toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood'], toronto_data['Borough']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#ff1493',
        fill=True,
        fill_color='#ff69b4',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

Now we get Scarborough's data from our overall data

In [6]:
scar_data = toronto_data[toronto_data['Borough'] == 'Scarborough']
scar_data

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


#### Finding venues located in the neighbourhoods of Scarborough

We define a function to get the different venues of all neighbourhoods using the Foursquare API

In [9]:
def get_all_venues(latitude, longitude, neighbourhood):
    
    final_dff = []
    
    CLIENT_ID ='WU2R1DNGLYATXFKLN5ITK0HXJEZR3YGYLXLUJTAY3VHPL53Z' 
    CLIENT_SECRET = '0PZG1UEPGZUWSYHONUIEOHIZVVC0R5EAVJ4MX4FNZM5DRFGZ' 
    ACCESS_TOKEN = 'ZDDLZJZUUWYMMU0151BLSD5BKBY5Z01D2P0LBO0UGV0VZ55Q' 
    VERSION = '20210806' 
    LIMIT = 100 
    radius = 500

    for lat, lng, ngh in zip(latitude, longitude, neighbourhood):
        neighborhood_latitude = lat
        neighborhood_longitude = lng
        neighborhood_name = ngh 
    
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        try:
            for ra in range(len(results)):
                row = {
                   'Neighbourhood':ngh,'Neighbourhood_Latitude':lat,'Neighbourhood_Longitude':lng,
                   'Venue':results[ra]['venue']['name'],'Venue_Latitude':results[ra]['venue']['location']['lat'],
                   'Venue_Longitude':results[ra]['venue']['location']['lng'],
                   'Venue_Category':results[ra]['venue']['categories'][0]['name']
               }
                final_dff.append(row)
                
        except Exception as err:
            print('Error occured:', err, ', skip row')
    
    ret_df = pd.DataFrame(final_dff)
    return ret_df

scar_venue_dataframe = get_all_venues(scar_data['Latitude'], scar_data['Longitude'], scar_data['Neighbourhood'])

In [10]:
scar_venue_dataframe

,Neighbourhood,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,"Malvern , Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood , Morningside , West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood , Morningside , West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
6,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
7,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
8,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Krispy Kreme Doughnuts,43.767169,-79.189660,Donut Shop
9,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center


Grouping by neighbourhood to show number of venues per neighbourhood

In [11]:
scar_venue_dataframe.groupby('Neighbourhood').count()

,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
Neighbourhood,,,,,,
Agincourt),5,5,5,5,5,5
"Birch Cliff , Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners , Tam O'Shanter , Sullivan",12,12,12,12,12,12
"Cliffside , Cliffcrest , Scarborough Village West",2,2,2,2,2,2
"Dorset Park , Wexford Heights , Scarborough Town Centre",7,7,7,7,7,7
"Golden Mile , Clairlea , Oakridge",10,10,10,10,10,10
"Guildwood , Morningside , West Hill",9,9,9,9,9,9
"Kennedy Park , Ionview , East Birchmount Park",6,6,6,6,6,6


We use one-hot encoding to analyze the neighbourhoods

In [12]:
scar_onehot = pd.get_dummies(scar_venue_dataframe[['Venue_Category']], prefix="", prefix_sep="")
scar_onehot['Neighbourhood'] = scar_venue_dataframe['Neighbourhood'] 
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]
scar_onehot

,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Donut Shop,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Gym Pool,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood , Morningside , West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
8,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
scar_onehot.shape

(94, 60)

In [14]:
scar_group = scar_onehot.groupby('Neighbourhood').mean().reset_index()
scar_group

,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Donut Shop,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Gym Pool,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt),0.0,0.000,0.000000,0.000000,0.000000,0.0,0.200000,0.000000,0.0,0.000000,0.0,0.00,0.000,0.000000,0.2,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.2,0.2,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.20,0.000000,0.0,0.000000,0.000000,0.000000
1,"Birch Cliff , Cliffside West",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.25,0.000,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.000000
2,Cedarbrae,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.125,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.125000,0.00,0.000000,0.125,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.125000,0.000000,0.000000
3,"Clarks Corners , Tam O'Shanter , Sullivan",0.0,0.000,0.000000,0.000000,0.083333,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000,0.083333,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.083333,0.083333,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.083333,0.083333,0.00,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.083333,0.000000,0.000000,0.083333,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.083333,0.000000,0.000000
4,"Cliffside , Cliffcrest , Scarborough Village West",0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000
5,"Dorset Park , Wexford Heights , Scarborough To...",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.142857,0.0,0.000000,0.0,0.00,0.000,0.142857,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.0,0.285714,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.142857,0.142857
6,"Golden Mile , Clairlea , Oakridge",0.0,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.000000,0.2,0.100000,0.0,0.00,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.1,0

In [15]:
scar_group.shape

(16, 60)

#### Finding the top 5 venues...

We run a loop to find the top 5 venues in each neighbourhood 

In [42]:
top_vens = 5

data_list = []

for hood in scar_group['Neighbourhood']:
    temp = scar_group[scar_group['Neighbourhood'] == hood].T.reset_index()   # transpose function
    temp.columns = ['Venue', 'Freq']                                         # Setting column names
    name = temp.iloc[0:1]['Freq'].item()                     # we capture the name of the neighbourhoods
    temp = temp.iloc[1:]                                     # removing the first row of new table
    temp['Freq'] = temp['Freq'].astype(float)                # cast frequence to float
    temp = temp.round({'Freq' : 2})                          # round to two decimal places
    result = temp.sort_values(by = 'Freq', ascending = False).reset_index(drop = True).head(top_vens)
    result = result.transpose()[0:1]
    result['Neighbourhood'] = name
    data_list.append(result)
   
popular_venues = pd.concat(data_list)
reset_cols = [popular_venues.columns[-1]] + list(popular_venues.columns[:-1])
popular_venues = popular_venues[reset_cols]
popular_venues.reset_index(drop = True, inplace = True)
popular_venues.columns = ['Neighbourhood', '1st Most Common', '2nd Most Common', '3rd Most Common', 
                          '4th Most Common', '5th Most Common']

In [43]:
popular_venues

,Neighbourhood,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common
0,Agincourt),Latin American Restaurant,Lounge,Breakfast Spot,Skating Rink,Clothing Store
1,"Birch Cliff , Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Mexican Restaurant
2,Cedarbrae,Caribbean Restaurant,Bakery,Bank,Thai Restaurant,Athletics & Sports
3,"Clarks Corners , Tam O'Shanter , Sullivan",Pizza Place,Italian Restaurant,Fried Chicken Joint,Intersection,Pharmacy
4,"Cliffside , Cliffcrest , Scarborough Village West",American Restaurant,Motel,Park,Intersection,Italian Restaurant
5,"Dorset Park , Wexford Heights , Scarborough To...",Indian Restaurant,Vietnamese Restaurant,Brewery,Chinese Restaurant,Pet Store
6,"Golden Mile , Clairlea , Oakridge",Bus Line,Bakery,Ice Cream Shop,Intersection,Metro Station
7,"Guildwood , Morningside , West Hill",Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
8,"Kennedy Park , Ionview , East Birchmount Park",Hobby Shop,Bus Station,Coffee Shop,Department Store,Discount Store
9,"Malvern , Rouge",Fast Food Restaurant,American Restaurant,Ice Cream Shop,Intersection,Italian Restaurant


### Clustering Neighbourhoods

We define our clusters

In [48]:
kclusters = 5

scar_cluster = scar_group.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 3, 1, 1, 1, 1, 2])

We join the neighbourhood data with the most popular venues, and add the clusters

In [50]:
 #popular_venues.insert(0, 'Cluster Labels', kmeans.labels_)
scarborough_merge = scar_venue_dataframe
scarborough_merge = scarborough_merge.join(popular_venues.set_index('Neighbourhood'), on='Neighbourhood')
scarborough_merge.head() 

,Neighbourhood,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category,Cluster Labels,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common
0,"Malvern , Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant,2,Fast Food Restaurant,American Restaurant,Ice Cream Shop,Intersection,Italian Restaurant
1,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,4,Bar,Moving Target,American Restaurant,Park,Intersection
2,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target,4,Bar,Moving Target,American Restaurant,Park,Intersection
3,"Guildwood , Morningside , West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
4,"Guildwood , Morningside , West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop


Then we create the map, and add our clusters to the map

In [53]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merge['Neighbourhood_Latitude'], 
                                  scarborough_merge['Neighbourhood_Longitude'], scarborough_merge['Neighbourhood'], 
                                  scarborough_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Tables for each cluster

#### Cluster 1

In [57]:
scarborough_merge.loc[scarborough_merge['Cluster Labels'] == 1, 
                      scarborough_merge.columns[[0] + [1] + list(range(5, scarborough_merge.shape[1]))]]

,Neighbourhood,Neighbourhood_Latitude,Venue_Longitude,Venue_Category,Cluster Labels,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common
3,"Guildwood , Morningside , West Hill",43.763573,-79.191151,Bank,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
4,"Guildwood , Morningside , West Hill",43.763573,-79.191537,Electronics Store,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
5,"Guildwood , Morningside , West Hill",43.763573,-79.191275,Restaurant,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
6,"Guildwood , Morningside , West Hill",43.763573,-79.190720,Mexican Restaurant,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
7,"Guildwood , Morningside , West Hill",43.763573,-79.193406,Rental Car Location,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
8,"Guildwood , Morningside , West Hill",43.763573,-79.189660,Donut Shop,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
9,"Guildwood , Morningside , West Hill",43.763573,-79.192286,Medical Center,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
10,"Guildwood , Morningside , West Hill",43.763573,-79.189490,Intersection,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
11,"Guildwood , Morningside , West Hill",43.763573,-79.190466,Breakfast Spot,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
12,Woburn,43.770992,-79.221156,Coffee Shop,1,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Ice Cream Shop,Intersection


#### Cluster 2

In [58]:
scarborough_merge.loc[scarborough_merge['Cluster Labels'] == 1, 
                      scarborough_merge.columns[[0] + [1] + list(range(5, scarborough_merge.shape[1]))]]

,Neighbourhood,Neighbourhood_Latitude,Venue_Longitude,Venue_Category,Cluster Labels,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common
3,"Guildwood , Morningside , West Hill",43.763573,-79.191151,Bank,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
4,"Guildwood , Morningside , West Hill",43.763573,-79.191537,Electronics Store,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
5,"Guildwood , Morningside , West Hill",43.763573,-79.191275,Restaurant,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
6,"Guildwood , Morningside , West Hill",43.763573,-79.190720,Mexican Restaurant,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
7,"Guildwood , Morningside , West Hill",43.763573,-79.193406,Rental Car Location,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
8,"Guildwood , Morningside , West Hill",43.763573,-79.189660,Donut Shop,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
9,"Guildwood , Morningside , West Hill",43.763573,-79.192286,Medical Center,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
10,"Guildwood , Morningside , West Hill",43.763573,-79.189490,Intersection,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
11,"Guildwood , Morningside , West Hill",43.763573,-79.190466,Breakfast Spot,1,Mexican Restaurant,Restaurant,Intersection,Bank,Donut Shop
12,Woburn,43.770992,-79.221156,Coffee Shop,1,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Ice Cream Shop,Intersection


#### Cluster 3

In [59]:
scarborough_merge.loc[scarborough_merge['Cluster Labels'] == 2, 
                      scarborough_merge.columns[[0] + [1] + list(range(5, scarborough_merge.shape[1]))]]

,Neighbourhood,Neighbourhood_Latitude,Venue_Longitude,Venue_Category,Cluster Labels,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common
0,"Malvern , Rouge",43.806686,-79.199056,Fast Food Restaurant,2,Fast Food Restaurant,American Restaurant,Ice Cream Shop,Intersection,Italian Restaurant


#### Cluster 4 

In [60]:
scarborough_merge.loc[scarborough_merge['Cluster Labels'] == 3, 
                      scarborough_merge.columns[[0] + [1] + list(range(5, scarborough_merge.shape[1]))]]

,Neighbourhood,Neighbourhood_Latitude,Venue_Longitude,Venue_Category,Cluster Labels,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common
42,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.240135,Motel,3,American Restaurant,Motel,Park,Intersection,Italian Restaurant
43,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.242353,American Restaurant,3,American Restaurant,Motel,Park,Intersection,Italian Restaurant


#### Cluster 5

In [61]:
scarborough_merge.loc[scarborough_merge['Cluster Labels'] == 4, 
                      scarborough_merge.columns[[0] + [1] + list(range(5, scarborough_merge.shape[1]))]]

,Neighbourhood,Neighbourhood_Latitude,Venue_Longitude,Venue_Category,Cluster Labels,1st Most Common,2nd Most Common,3rd Most Common,4th Most Common,5th Most Common
1,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.163085,Bar,4,Bar,Moving Target,American Restaurant,Park,Intersection
2,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.162977,Moving Target,4,Bar,Moving Target,American Restaurant,Park,Intersection
